# Instructions

This colab contains code used to run the ridgeplot visualization for the Independent Trait Shaping experiments on the various LLMs experimented on in the paper "Personality Traits in Large Language Models" (https://arxiv.org/pdf/2307.00184). The code assumes that all the data produced and consumed in the colab (especially the pickled dataframe outputs of running inference on various LLMs) lives in a local filesystem either in a cloud instance running a Jupyter notebook such as Google Colab or a desktop. But those file I/O operations can easily be replaced to use any other file management solutions. The inline comments for some of the operations explain the motivation behind them and what to expect in the results of running an analysis in a cell.

To run this colab:
1. Connect to an appropriate runtime. (For instance, if running the bulk inference directly from the colab, connect to a GPU kernel.)
2. Check experiment parameters below.
3. Run the code cells for visualizations.

NOTE: Make sure to store and run this notebook from a location where the Psyborgs codebase package is stored (personality_in_llms.psyborgs)

# Setup
The repo containing this notebook has a version of the Psyborgs codebase needed to make the notebook run. But in case a more recent version is needed, it can be fetched from https://github.com/google-research/google-research/tree/master/psyborgs.

### Install Psyborgs Dependencies

In [ ]:
#@markdown Run this cell to install the dependencies needed to run Psyborgs.
#@markdown The dependencies are in a requirements.txt file in the Psyborgs repo.
%pip install -r personality_in_llms/psyborgs/requirements.txt

In [ ]:
#@title Load Libraries
#@markdown Run this cell to import dependencies
import pandas as pd
import numpy as np

from psyborgs import score_calculation
from psyborgs import survey_bench_lib

# dependencies for descriptive statistics
import itertools
from typing import Union, List

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import scipy

In [ ]:
#@title File locations setup  { run: "auto" }

#@markdown `PKL_PATH` is the filename of pickled results to be analyzed.
#@markdown This is the output dataframes from the LLM inference runs packaged in pkl format.
#@markdown This is the input for this colab.
PKL_PATH = 'sample_pkl_file.pkl'  # @param {"type":"string"}

#@markdown `ADMIN_SESSION_PATH` is the file path of the input admin session that the experiment is based on.
ADMIN_SESSION_PATH = 'admin_sessions/sample_admin_session.json'  # @param {"type":"string"}

#@markdown Path of the file where the joined test scores pickled dataframe should be stored.
SAVE_SCORES_FILENAME = 'sample_scored_dataframe.pkl'  # @param {"type":"string"}

#@markdown Whether the model who's data is being analyzed is a PaLM model variant or not?
#@markdown Some of the pre-processing on the input dataframe differs between PaLM and non-PaLM models.
IS_PALM_MODEL = True  # @param {"type":"boolean"}

#@markdown This is a model identifier needed for Psyborgs code. More info here: psyborgs/survey_bench_lib.py:L63
MODEL_ID = 'PaLM'  #@param {type:"string"}


In [ ]:
SPID = ['item_preamble_id',
        'item_postamble_id',
        'response_scale_id',
        'response_choice_postamble_id',
        'model_id']

BFI_SCALE_IDS = ['BFI-EXT', 'BFI-AGR', 'BFI-CON', 'BFI-NEU', 'BFI-OPE']
IPIP_SCALE_IDS = ['IPIP300-EXT', 'IPIP300-AGR', 'IPIP300-CON', 'IPIP300-NEU', 'IPIP300-OPE']
VALIDATION_SCALE_IDS = ['PA', 'NA', 'CSE', 'CPI', 'PHYS', 'VRBL', 'ANGR', 'HSTL', 'ACHV', 'CONF', 'SCRT']

## Unpickle Raw Results

In [ ]:
df_raw_response_scores = pd.read_pickle(PKL_PATH)

# if PaLM model inference was used, convert from byte to string
if IS_PALM_MODEL:
  for col, dtype in df_raw_response_scores.dtypes.items():
    if dtype == object:  # Only process byte object columns.
      df_raw_response_scores[col] = df_raw_response_scores[col].apply(lambda x: x.decode('utf-8'))

In [ ]:
df_raw_response_scores.head(5)

In [ ]:
test_df = df_raw_response_scores.query(
    "item_postamble_id == 'plk-ipip-0' & item_preamble_id == 'ext0-agr2-con0-neu0-ope0-d36-ev2' & item_id == 'ipip1'"
)

test_df

## Load Admin Session


In [ ]:
admin_session = survey_bench_lib.load_admin_session(
    ADMIN_SESSION_PATH)

# Score Session

In [ ]:
if not IS_PALM_MODEL:
  # adapt df to match a df with scores for possible continuations
  df_raw_response_scores['score'] = 1
  df_raw_response_scores['response_value'] = df_raw_response_scores['model_output'].astype('int')

In [ ]:
# score session
scored_session_df = score_calculation.score_session(
    admin_session, df_raw_response_scores)

scored_session_df.head(5)

In [ ]:
# optional: save scores to disk
if SAVE_SCORES_FILENAME:
  scored_session_df.to_pickle(SAVE_SCORES_FILENAME)

# Ridgeplots

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
def get_domain_fragments(big5_id, levels=range(1, 10)):
  """Returns list of preamble ID fragments for one domain."""
  return [f'{big5_id}{i}' for i in levels]


def get_big5_lvl_fragments(levels=range(1, 10)):
  """Returns list of preamble ID fragments for all Big Five domains."""
  big5_id_fragments = ['ext', 'agr', 'con', 'neu', 'ope']
  nested_fragments = [get_domain_fragments(big5_id, levels) for big5_id in big5_id_fragments]
  preamble_id_fragments = list(itertools.chain(*nested_fragments))
  return preamble_id_fragments


def subset_one_preamble(df, id_fragment):
  return df[df['item_preamble_id'].str.contains(id_fragment)][IPIP_SCALE_IDS]


def subset_by_preambles(df, id_fragments):
  """Subsets data by a given list of item preamble fragments."""
  preambles = []

  for id_fragment in id_fragments:
    preambles.append(subset_one_preamble(df, id_fragment))

  return pd.concat(preambles, keys=id_fragments)


def describe_by_preambles(df,
                          id_fragments,
                          by: Union[str, List[str]] = ['median', 'min', 'max', 'std']):
  """Describe dataframe using summary statistics grouping by preambles."""
  # organize data by preamble_id fragment
  df_by_preambles = subset_by_preambles(df, id_fragments)

  # group by preamble_id fragments
  df_grouped = df_by_preambles.groupby(level=0)

  # aggregate by specified summary stats
  summary = df_grouped.agg(by)

  return summary

In [ ]:
PLOT_SPACE = np.linspace(1., 5.1)
PLOT_COLUMNS = ['IPIP300-EXT', 'IPIP300-AGR', 'IPIP300-CON', 'IPIP300-NEU', 'IPIP300-OPE']
EXT_COUNT = 10

DIMENSION_PREFIXES = ['ext', 'agr', 'con', 'neu', 'ope']

fig = make_subplots(rows=5, cols=len(PLOT_COLUMNS),
                    shared_xaxes=True, shared_yaxes=True,
                    column_titles=['IPIP-NEO EXT', 'IPIP-NEO AGR', 'IPIP-NEO CON', 'IPIP-NEO NEU', 'IPIP-NEO OPE'],
                    row_titles=['Prompted EXT', 'Prompted AGR', 'Prompted CON', 'Prompted NEU', 'Prompted OPE'],
                    x_title='Observed Personality Scores',
                    vertical_spacing=0.01, horizontal_spacing=0.01)
big5_domain_lvls = get_big5_lvl_fragments()
sub_pre_df = subset_by_preambles(scored_session_df, big5_domain_lvls)

y_ticks_coordinates = []

for p, prefix in enumerate(DIMENSION_PREFIXES):
  for i, (ext_id, sub_df) in enumerate(sub_pre_df.groupby(level=0)):
    if prefix not in ext_id: continue
    for j, plot_col in enumerate(PLOT_COLUMNS):
      dist_to_plot = sub_df[plot_col]
      counts, bins, _ = scipy.stats.binned_statistic(dist_to_plot.values, values=None, statistic='count', bins=PLOT_SPACE, range=(1., 5.1))

      scatter_plot = go.Scatter(
          x=np.concatenate([np.array([1.]), bins]),
          y=np.concatenate([np.array([i*5.]), counts + i*5]),
          fill='toself',
          mode='lines+text')


      fig.add_trace(scatter_plot, row=p+1, col=j+1)

    y_ticks_coordinates.append(scatter_plot.y.min())

fig.update_layout(width=1024, height=1024, showlegend=False)

y_ticks_coordinates = out = [y_ticks_coordinates[i: i+9] for i in range(0, len(y_ticks_coordinates), 9)]
y_ticks = [
    ('yaxis', 'ext', y_ticks_coordinates[0]),
    ('yaxis6', 'agr', y_ticks_coordinates[1]),
    ('yaxis11', 'con', y_ticks_coordinates[2]),
    ('yaxis16', 'neu', y_ticks_coordinates[3]),
    ('yaxis21', 'ope', y_ticks_coordinates[4]),
]

for axis_id in ['xaxis21', 'xaxis22', 'xaxis23', 'xaxis24', 'xaxis25']:
  fig['layout'][axis_id].update({
      'tickfont': dict(size=16)
  })

for axis_id, tick_id, coordinates in y_ticks:
  fig['layout'][axis_id].update({
      'showticklabels': True, 'visible': True,
      'tickmode': 'array',
      'tickvals': coordinates,
      'ticktext': ['1', '', '3', '', '5', '', '7', '', '9'],
      'tickfont': dict(size=16)
  })

fig.for_each_annotation(lambda a: a.update(font=dict(size=(22 if a['textangle'] != 90 else 20))))
fig.show()

In [ ]:
def plot_ridge(prefix, title, axis_label):
  f = make_subplots(rows=1, cols=len(PLOT_COLUMNS),
                    shared_xaxes=True, shared_yaxes=True,
                    column_titles=PLOT_COLUMNS,
                    row_titles=[axis_label])
  sub_pre_df = subset_by_preambles(scored_session_df, big5_domain_lvls)
  for i, (ext_id, sub_df) in enumerate(sub_pre_df.groupby(level=0)):
    if prefix not in ext_id: continue
    for j, plot_col in enumerate(PLOT_COLUMNS):
      dist_to_plot = sub_df[plot_col]
      counts, bins, _ = scipy.stats.binned_statistic(dist_to_plot.values, values=None, statistic='count', bins=PLOT_SPACE, range=(1., 5.1))

      plot = go.Scatter(
          x=bins,
          y=counts + i*5,
          fill='toself',
          mode='lines')

      f.add_trace(plot, row=1, col=j+1)

  f.update_layout(width=800, height=400, showlegend=False, title=title)

  f.update_layout(
      yaxis=dict(
          nticks=EXT_COUNT,
          tickmode='array',
          tickvals=[i*15 for i in range(EXT_COUNT)],
          ticktext=[i for i in range(EXT_COUNT)]
      )
  )

  f.show()

In [ ]:
plot_ridge(DIMENSION_PREFIXES[0], title='Distribution of response scores when increasing levels of extroversion', axis_label='Extroversion level')

In [ ]:
plot_ridge(DIMENSION_PREFIXES[1], title='Distribution of response scores when increasing levels of agreeableness', axis_label='Agreeableness level')

In [ ]:
plot_ridge(DIMENSION_PREFIXES[2], title='Distribution of response scores when increasing levels of conscientiousness', axis_label='Conscientiousness level')

In [ ]:
plot_ridge(DIMENSION_PREFIXES[3], title='Distribution of response scores when increasing levels of neuroticism', axis_label='Neuroticism level')

In [ ]:
plot_ridge(DIMENSION_PREFIXES[4], title='Distribution of response scores when increasing levels of openness', axis_label='Openness level')